In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import os
import json
import torch.nn as nn
import math
from tqdm.auto import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

0 NVIDIA GeForce RTX 3090


In [3]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [4]:
batch_size = 1
n_epochs = 500
model_no = 'transformer_with_word2vec_without_zeros_and_labels'
exp = 3

In [5]:
class custom_transformer(nn.Module):

    def __init__(self, hidden_dim=100, nheads=5, num_encoder_layers=8, num_decoder_layers=8, avg_words = 5500):
        super(custom_transformer, self).__init__()
#         self.transformer = nn.Transformer(hidden_dim, nheads, num_encoder_layers, num_decoder_layers, 
#                                           batch_first=True, activation="relu")
        self.encoder = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nheads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder, num_layers = num_encoder_layers)
        
        self.decoder = nn.TransformerDecoderLayer(d_model=hidden_dim, nhead=nheads, batch_first=True)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder, num_layers = num_decoder_layers)
        
#         self.linear1 = nn.Linear(hidden_dim*avg_words, hidden_dim*avg_words, bias=True)
#         self.one_D_conv = nn.Conv2d(hidden_dim*avg_words, hidden_dim*avg_words, 1, stride=1)
#         self.relu_layer = nn.ReLU()
#         self.sigmoid_layer = nn.Sigmoid()
        
    def positionalencoding1d(self, d_model, length):
        """
        :param d_model: dimension of the model
        :param length: length of positions
        :return: length*d_model position matrix
        """
        if d_model % 2 != 0:
            raise ValueError("Cannot use sin/cos positional encoding with "
                             "odd dim (got dim={:d})".format(d_model))
        pe = torch.zeros(length, d_model)
        position = torch.arange(0, length).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                             -(math.log(10000.0) / d_model)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)

        return pe

    def forward(self, feat_input):
#         print(feat_input.shape)
#         feat_input = feat_input.flatten(2)
#         print(feat_input.shape)
#         print(self.positionalencoding1d(self.hidden_dim, feat_input.shape[-2]).repeat(feat_input.shape[0], feat_input.shape[1], feat_input.shape[2]).is_cuda)
#         feat_input += self.positionalencoding1d(self.hidden_dim, feat_input.shape[-2]).repeat(feat_input.shape[0], feat_input.shape[1], feat_input.shape[2])
#         features = self.transformer(feat_input.cuda(), self.learnable_query.repeat(feat_input.shape[0], 1, 1))
        enc_features = self.transformer_encoder(feat_input)
            
        dec_features = self.transformer_decoder(feat_input, enc_features)

#         features = self.linear1(dec_features.flatten(1))
#         dec_features = self.sigmoid_layer(features)

        return dec_features, enc_features

In [6]:
new_model = custom_transformer()
# new_model.to(device)
print("Total trainable params:", torch.nn.utils.parameters_to_vector([p for p in new_model.parameters() if p.requires_grad]).numel())

Total trainable params: 8511264


/home/abhijeet/miniconda3/envs/TRIZ/lib/python3.8/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
new_model

custom_transformer(
  (encoder): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (linear1): Linear(in_features=100, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=100, bias=True)
    (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
    

In [8]:
# Removed Stop Words

In [9]:
import json
file_text = open('/home/abhijeet/Desktop/TRIZ/word_vectors_all_words_v2.json', 'r')
dictionary_text = json.load(file_text)
# file_label = open('/home/abhijeet/Desktop/TRIZ/labels.json', 'r')
# dictionary_label = json.load(file_label)

In [10]:
# DataLoader with zeros being appended
# from torch.utils.data import Dataset, DataLoader

# class dataset_loader(Dataset):
    
#     def __init__(self, corpus_dir, word2vec_text, word2vec_label):
#         self.word2vec_all_text = word2vec_text
#         self.word2vec_all_label = word2vec_label
#         self.dataset = pd.read_excel(corpus_dir + 'generated_data.xlsx')
#         self.dataset = self.dataset.dropna()
#         self.cols = self.dataset.columns
#         self.dataset = self.dataset.values
#         self.dataset = pd.DataFrame(self.dataset, columns = cols)
#         self.text = self.dataset['text']
#         self.label = self.dataset['labels']
        
#     def __len__(self):
#         return self.text.shape[0]
    
#     def __getitem__(self, idx):
#         row_text = self.text[idx].lower().replace('\n', ' ')
#         row_label = eval(self.label[idx])
#         stop_words = set(stopwords.words('english'))
#         word_tokens = word_tokenize(row_text)
#         filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#         filtered_sentence = []
        
#         for w in word_tokens:
#             if w not in stop_words:
#                 filtered_sentence.append(w)
        
#         row_text = filtered_sentence
        
# #         print(row_label, type(row_label))
#         word2vec_matrix = []
# #         print(len(row_text))
#         for count, i in enumerate(row_text):
#             try:
#                 if count == 5000:
#                     break
#                 word2vec_matrix.append(np.array(self.word2vec_all_text[i]))
# #                 print('Doing Text', count)
#             except:
#                 if count == 5000:
#                     break
#                 word2vec_matrix.append(np.zeros(100))
        
#         for i in range(5000 - len(word2vec_matrix)):
#             word2vec_matrix.append(np.zeros(100))
            
# #         print(np.array(word2vec_matrix).shape)

#         for next_count, i in enumerate(row_label):
#             try:
#                 if next_count == 20:
#                     break
#                 word2vec_matrix.append(np.array(self.word2vec_all_label[i]))
# #                 print('Doing Labels')
#             except:
#                 if next_count == 20:
#                     break
#                 word2vec_matrix.append(np.zeros(100))
            
#         for i in range(5020 - len(word2vec_matrix)):
#             word2vec_matrix.append(np.zeros(100))

# #         print(np.array(word2vec_matrix).shape)
        
#         output = {'text_label': np.array(word2vec_matrix)}
        
#         if np.array(word2vec_matrix).shape != (5020, 100):
#             print(np.array(word2vec_matrix).shape)
#             output = {'text_label': np.array(word2vec_matrix[:5020])}
#         return output

In [11]:
from torch.utils.data import Dataset, DataLoader

class dataset_loader(Dataset):
    
    def __init__(self, corpus_dir, word2vec_text):
        self.word2vec_all_text = word2vec_text
#         self.word2vec_all_label = word2vec_label
        self.dataset = pd.read_excel(corpus_dir + 'generated_data.xlsx')
        self.dataset = self.dataset.dropna()
        self.cols = self.dataset.columns
        self.dataset = self.dataset.values
        self.dataset = pd.DataFrame(self.dataset, columns = self.cols)
        self.text = self.dataset['text']
        self.label = self.dataset['labels']
        
    def __len__(self):
        return self.text.shape[0]
    
    def __getitem__(self, idx):
        row_text = self.text[idx].lower().replace('\n', ' ')
        row_label = eval(self.label[idx])
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(row_text)
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        filtered_sentence = []
        
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        
        row_text = filtered_sentence
        
#         print(row_label, type(row_label))
        word2vec_matrix = []
#         print(len(row_text))
        for count, i in enumerate(row_text):
            try:
                word2vec_matrix.append(np.array(self.word2vec_all_text[i]))
#                 print('Doing Text', count)
            except:
                word2vec_matrix.append(np.zeros(100))

#         print(np.array(word2vec_matrix).shape)
        
        output = {'text_label': np.array(word2vec_matrix)}

        return output

In [12]:
file_text.close()
# file_label.close()
train_data = dataset_loader(corpus_dir = '/home/abhijeet/Desktop/TRIZ/All_data/CPC Data/',
                              word2vec_text = dictionary_text)

In [13]:
# print(len(train_data))
for i in range(len(train_data)):
    sample = train_data[1]
    break

In [14]:
sample['text_label'].shape

(157, 100)

In [15]:
train_sampler = torch.utils.data.RandomSampler(train_data)

dataloader_train = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=0)

In [16]:
# import torch.nn.functional as F
# def criterion(predicted, target):
#     """
#     Compute the Kullback-Leibler Divergence loss between two probability distributions.

#     Args:
#         p (torch.Tensor): True distribution (e.g., ground truth probabilities).
#         q (torch.Tensor): Approximate distribution (e.g., predicted probabilities).

#     Returns:
#         torch.Tensor: KL Divergence loss.
#     """
#     assert target.shape == predicted.shape, "Target and predicted tensors must have the same shape"

#     # Apply log softmax to the target and softmax to the predicted
#     log_target = F.log_softmax(target, dim=1)
#     softmax_predicted = F.softmax(predicted, dim=1)

#     # Compute the KL divergence loss
#     loss = F.kl_div(log_target, softmax_predicted, reduction='batchmean')

#     return loss

In [17]:
import torch.nn.functional as F
def criterion(predicted, target):
    
    loss = nn.MSELoss()

    return loss(predicted, target)

In [18]:
new_model.to(device)
optimizer = torch.optim.SGD(new_model.parameters(), lr=0.009, weight_decay=0.0001, momentum=0.9)
# optimizer = torch.optim.Adam(new_model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [19]:
for epoch in range(n_epochs):
    num_nans = 0
    running_loss = 0.0
    val_loss = 0.0
    print(f"\nEpoch: {epoch+1}")
    inner_pbar = tqdm(total=len(dataloader_train), position=1, leave=False, ascii=True, desc=f"Epoch: {epoch+1} Step")
#     try:
    for i, data in enumerate(dataloader_train, 1):
        features = data['text_label']
        features = features.type(torch.FloatTensor)
        features = features.to(device)
#         print(features.is_cuda)
#         sigmoid = nn.Sigmoid()
        y_ground_truth = torch.tensor(features, requires_grad = True)
#         print(features.shape, y_ground_truth.shape)
        optimizer.zero_grad()
        preds, enc_results = new_model(features)

        loss = criterion(predicted=preds, target=y_ground_truth)

        inner_pbar.update(1)

        running_loss += loss.detach().item()

        print(f"\rStep: {i} Training Loss: {loss.item()} Validation Loss: {val_loss} Epoch loss: {running_loss/(i)}", end="")
        # print(f"\rStep: {i} Training Loss: {loss.item()} Validation Loss: {val_loss} Nans: {num_nans}")

        if torch.isnan(loss):
            if num_nans > 10:
                raise RuntimeError(f"Model Error: Encountered {num_nans} nan loss")
            num_nans += 1
            continue
        # loss.requires_grad = True
        loss.backward()

        optimizer.step()

        if (i % 500 == 0):
            if os.path.exists(f'/home/abhijeet/Desktop/TRIZ/Model/model_{model_no}/'):
                pass
            else:
                os.makedirs(f'/home/abhijeet/Desktop/TRIZ/Model/model_{model_no}/exp_{str(exp)}/')

            checkpoint_pth = f'/home/abhijeet/Desktop/TRIZ/Model/model_{model_no}/exp_{str(exp)}/trained_model_latest_epoch.pth'
            torch.save(new_model.state_dict(), checkpoint_pth)

    scheduler.step()
    inner_pbar.close()
#     except:
#         if os.path.exists(f'/home/abhijeet/Desktop/TRIZ/Model/model_{model_no}/'):
#             pass
#         else:
#             os.makedirs(f'/home/abhijeet/Desktop/TRIZ/Model/model_{model_no}/exp_{str(exp)}/')

#         checkpoint_pth = f'/home/abhijeet/Desktop/TRIZ/Model/model_{model_no}/exp_{str(exp)}/trained_model_latest_epoch.pth'
#         torch.save(new_model.state_dict(), checkpoint_pth)


Epoch: 1


Epoch: 1 Step:   0%|          | 0/173749 [00:00<?, ?it/s]

/tmp/ipykernel_4507/2807553944.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_ground_truth = torch.tensor(features, requires_grad = True)


Step: 173749 Training Loss: 0.036992594599723816 Validation Loss: 0.0 Epoch loss: 0.06882871317364987
Epoch: 2


Epoch: 2 Step:   0%|          | 0/173749 [00:00<?, ?it/s]

...

Epoch: 14 Step:   0%|          | 0/173749 [00:00<?, ?it/s]

Step: 173749 Training Loss: 0.03596973046660423 Validation Loss: 0.0 Epoch loss: 0.0460804376615958365
Epoch: 15


Epoch: 15 Step:   0%|          | 0/173749 [00:00<?, ?it/s]

Step: 36416 Training Loss: 0.033448897302150726 Validation Loss: 0.0 Epoch loss: 0.046266531179214995

KeyboardInterrupt: 

In [20]:
if os.path.exists(f'/home/abhijeet/Desktop/TRIZ/Model/model_{model_no}/'):
    pass
else:
    os.makedirs(f'/home/abhijeet/Desktop/TRIZ/Model/model_{model_no}/exp_{str(exp)}/')

checkpoint_pth = f'/home/abhijeet/Desktop/TRIZ/Model/model_{model_no}/exp_{str(exp)}/trained_model_latest_epoch.pth'
torch.save(new_model.state_dict(), checkpoint_pth)